Integrated Local Threats to Coral Reefs	bio.024.5 http://www.wri.org/publication/reefs-risk-revisited

In [6]:
import numpy as np
import pandas as pd
import rasterio as rio

import boto3
import requests as req

from matplotlib import pyplot as plt
%matplotlib inline
import os
import sys
import threading

In [2]:
s3_bucket = "wri-public-data"
s3_key_orig = "resourcewatch/raster/bio_024_5_coral_reef_integrated_local_threats/bio_024_5_coral_reef_integrated_local_threats.tif"

temp_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/"
local_orig = temp_folder + "bio_024_5.tif"

s3_upload = boto3.client('s3')
s3_download = boto3.resource('s3')

#s3.meta.client.download_file(s3_bucket, s3_key_orig, local_orig)

In [3]:
s3_key_edit = s3_key_orig[:-4] + "_edit.tif"

In [4]:
def reproject_file(orig_name, edit_name):
    with rio.open(orig_name, 'r') as src:
        # This assumes data is readable by rasterio
        # May need to open instead with netcdf4.Dataset, for example
        
        srcprofile = src.profile
        rows = srcprofile["height"]
        columns = srcprofile["width"]

        print(rows)
        print(columns)

        # Latitude bounds
        south_lat = -90
        north_lat = 90

        # Longitude bounds
        west_lon = -180
        east_lon = 180

        transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)

        # Profile

        no_data_val = srcprofile["nodata"]
        target_projection = 'EPSG:4326'
        target_data_type = np.float64

        profile = {
            'driver':'GTiff', 
            'height':rows, 
            'width':columns, 
            'count':1, 
            'dtype':target_data_type, 
            'crs':target_projection, 
            'transform':transform, 
            'compress':'lzw', 
            'nodata': no_data_val
        }

        windows = src.block_windows()
        with rio.open(edit_name, "w", **profile) as dst:
            for ix, window in windows:
                data = src.read(1, window=window)
                dst.write(data.astype(profile["dtype"]), 1, window=window)

In [7]:
local_edit = temp_folder + "bio_024_5_edit.tif"

reproject_file(local_orig, local_edit)

25456
80150


In [8]:
# Helper function to view upload progress
class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [10]:
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit, Callback=ProgressPercentage(local_edit))

/Users/nathansuberi/Desktop/RW_Data/Rasters/bio_024_5_edit.tif  123929865 / 123929865.0  (100.00%)